### Библиотеки / данные

импортируем библиотеки numpy и pandas

In [ ]:
import numpy as np
import pandas as pd

импортируем библиотеку matplotlib для построения графиков

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

задаем некоторые настройки pandas, регулирующие формат вывода

In [ ]:
pd.options.display.max_rows = 10

- считываем данные 
- используем столбец Symbol в качестве индекса 
- считываем только те столбцы ['Symbol', 'Sector', 'Price', 'Book Value', 'Market Cap', 'Dividend Yield']

| Column Name        | Description
| ------------- |:-------------:|
|Symbol|Сокращенное название организации|
|Name|Полное название организации|
|Sector|Сектор экономики|
|Price|Стоимость акции|
|Dividend Yield|Дивидендная доходность|
|Price/Earnings|Цена / прибыль|
|Earnings/Share|Прибыль на акцию|
|Book Value|Балансовая стоимость компании|
|52 week low|52-недельный минимум|
|52 week high|52-недельный максимум|
|Market Cap|Рыночная капитализация|
|EBITDA|**E**arnings **b**efore **i**nterest, **t**axes, **d**epreciation and **a**mortization|
|Price/Sales|Цена / объём продаж|
|Price/Book|Цена / балансовая стоимость|
|SEC Filings|Ссылка *sec.gov*|

In [ ]:
sp500 = pd.read_csv("../data/sp500.csv",
                    index_col='Symbol',
                    usecols=['Symbol', 'Sector', 'Price',
                             'Book Value', 'Market Cap',
                             'Dividend Yield'])
sp500

считываем исторические данные о котировках акций

In [ ]:
omh = pd.read_csv('../data/omh.csv', 
                  parse_dates=['Date'])

omh.set_index('Date', 
              inplace=True)

In [ ]:
omh.head()

### Настройка 

#### пример 

график цены акции Microsoft

In [ ]:
omh.MSFT.plot()

цена акций microsoft и apple на одном графике

In [ ]:
omh.plot()

нормализация:

In [ ]:
omh_copy =  (omh - omh.mean())/omh.std()
omh_copy.plot()

#### размер 

увеличиваем размер графика

In [ ]:
omh_copy.plot(figsize=(10, 5))

#### заголовок / подписи

- заголовок задаем с помощью параметра title метода .plot()
- после вызова метода .plot() явно задаем подписи осей x и y

In [ ]:
omh_copy.plot(title='Цены акций после нормировки')
plt.xlabel('Дата')
plt.ylabel('Цена')

#### легенда

изменяем элементы легенды, соответствующие именам столбцов датафрейма

In [ ]:
ax = omh_copy.plot()
ax.legend(['Microsoft', 'Apple'])

изменяем расположение легенды

In [ ]:
ax = omh_copy.plot()
ax.legend(loc='upper center')

отключаем легенду с помощью legend=False

In [ ]:
omh_copy.plot(legend=False)

#### цвет / стиль / толщина / маркер

меняем цвета линий графика  
[односимвольные коды](https://matplotlib.org/2.0.2/api/colors_api.html)  
[шестнадцатеричный RGB-код](https://www.rapidtables.com/web/color/RGB_Color.html)

In [ ]:
omh_copy.plot(style={'MSFT': '#0000FF', 'AAPL': 'g'}) 

- [различные стили линий](https://matplotlib.org/1.5.3/api/pyplot_api.html#matplotlib.pyplot.plot)  
- увеличиваем толщину линий

In [ ]:
omh_copy.plot(style={'MSFT': 'b--', 'AAPL': 'g:'}, lw=3)

рисуем заново, добавив [маркеры](https://matplotlib.org/api/markers_api.html) линий

In [ ]:
omh_copy.plot(style={'MSFT': 'b--^', 'AAPL': 'g:o'})

### Графики

#### bar

строим столбчатую диаграмму

In [ ]:
s = sp500.Sector.value_counts()
s.plot(kind='bar')

малочисленные сектора:

In [ ]:
small_sectors = s[-4:].index.values
small_sectors

95% квантиль

In [ ]:
sp500.quantile(0.95)

создаем копию данных sp500 

In [ ]:
ind = (~sp500.Sector.isin(small_sectors)) \
    & (sp500.Price < 184) \
    & (sp500['Book Value'] < 66) \
    & (sp500['Market Cap'] < 134) \
    & (sp500['Dividend Yield'] < 4.5)

In [ ]:
sp500_cut = sp500.loc[ind].copy()

In [ ]:
sp500_cut.shape

строим столбчатую диаграмму на основе нескольких серий

In [ ]:
df = sp500_cut.groupby('Sector').Price.agg(['mean', 'median'])
df.plot(kind='bar', figsize=(10, 5))

In [ ]:
df

вертикальная состыкованная столбчатая диаграмма

In [ ]:
df.plot(kind='bar', stacked=True, figsize=(10, 5));

горизонтальная состыкованная столбиковая диаграмма

In [ ]:
df.plot(kind='barh', stacked=True, figsize=(10, 5));

#### hist

строим гистограмму

In [ ]:
sp500_cut.Price.hist()

снова строим гистограмму, но теперь с большим количеством интервалов (бинов)

In [ ]:
sp500_cut.Price.hist(bins = 50);

поскольку у нас 4 столбца, pandas строит четыре гистограммы

In [ ]:
sp500_cut.hist(figsize=(12,8))

напрямую воспользуемся модулем pyplot  для наложения нескольких гистограмм

In [ ]:
plt.figure(figsize=(12, 6))
plt.hist(sp500_cut.Price, alpha=0.5, label='Price')
plt.hist(sp500_cut['Book Value'], alpha=0.5, label='Book Value')
plt.legend(loc='upper right');

In [ ]:
s = sp500_cut.Price
s.hist(density=True)
s.plot(kind='kde', figsize=(10,6))

#### box

<img src='..\images\boxplot.png'>

In [ ]:
sp500_cut[['Price','Book Value']].boxplot(figsize = (14, 7))

#### scatter

In [ ]:
sp500_cut.plot(kind='scatter', x='Price', y='Book Value', figsize=(10, 6));

In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(sp500_cut, alpha=0.4, figsize=(9, 9), diagonal='kde');

#### heat map

In [ ]:
corr_matrix = sp500_cut.corr()
corr_matrix

In [ ]:
plt.figure(figsize=(7, 7))
plt.imshow(corr_matrix, cmap='Greens')
plt.colorbar()  # добавим шкалу интенсивности цвета

plt.xticks(range(len(corr_matrix.columns)), corr_matrix.columns)
plt.yticks(range(len(corr_matrix)), corr_matrix.index);